<a href="https://colab.research.google.com/github/SnSabu/congressional-analysis/blob/main/Flight_Data_Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Webscraping using Beautiful Soup

In [8]:
import time
import requests
from bs4 import BeautifulSoup
origin = "LAX"
destination = "SFO"
start_date = "2023-03-19"
base_url = "https://www.kayak.com/flights"
url = f"{base_url}/{origin}-{destination}/{start_date}?sort=bestflight_a"
url
response = requests.get(base_url)
time.sleep(1)
soup = BeautifulSoup(response.content, 'html.parser')
soup

<!DOCTYPE html>

<html class=" Flights-Search-FlightSearchPage-Html" lang="en-us">
<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#">
<link data-type="page" href="https://content.r9cdn.net/res/combined.css?v=e14f0d47c8e1efcbf67f83ad1c0a98ffd2643893-14pre-flipped&amp;cluster=4" rel="stylesheet"/>
<link data-type="page" href="https://content.r9cdn.net/res/combined.css?v=dbef17b650aa59926cb6c786afd8f3d152d78f92-14pre-flipped&amp;cluster=4" rel="stylesheet"/>
<link data-type="page" href="https://content.r9cdn.net/res/css/stl.css?v=a9dc31451ced7514155ce57fd4f63fcb7ce87c4c-14pre-flipped-nc&amp;cluster=4" rel="stylesheet"/>
<style data-type="style-metadata2" type="application/metadata2">
28Bg5E86Q1Q13C1I29C37Q14g13M51B50Q189g117g96FC/C8K4I38C6C2ES1g44E312SZIkSZIkSZIkSRIkSZIkSQ1kSZIkSRI115Ik1R1kCQ3B13g3gCQ1E9B1k140C171QJIkC99SZIkSRIgQRI16kSZIkSZIk1JIECQ93gSZI2JIkSJIkSRI1SQI2YIESRI8gQYI1CYIESI2SY1kQQI90Q2QZIkSZ6QJIkSZI2B18SQ2C3SQ1kSIIkSZIkSQ5gSRI530JI13QJIkSR253kSQI=</style>
<meta c

In [ ]:
from selenium.common.exceptions import NoSuchElementException

# Close cookies pop-up
accept_cookies_xpath = '/html/body/div[12]/div/div[3]/div/div/div/div/div[1]/div/div[2]/div[2]/div[1]/button/span'

try:
   browser.find_element_by_xpath(accept_cookies_xpath).click()
except NoSuchElementException:
   pass

Webscraping using selenium

In [ ]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-browser
!apt install chromium-chromedriver
from selenium import webdriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 7.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--diable-dve-shm-uage')

driver = webdriver.Chrome(executable_path='/usr/lib/chromium-browser/chromedriver', options = options)

<ipython-input-4-c5c057d8dd05>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/usr/lib/chromium-browser/chromedriver', options = options)


WebDriverException: ignored